In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import sys 
import datetime as dt
import time
import gc

In [1]:
from helper.parallel_util import ParallelUtil
from helper.parsers import Utils
from helper.file_mapping import FILE_MAPPINGS
from helper.imdb_sqllite_db import IMDBSQLLite
import config


LESSON_DATA_FOLDER = config.IMDB_DATA_FOLDER
imdb_db = IMDBSQLLite();
imdbConn, imdbCurs = imdb_db.get()

IMDB SQLLite Database: ../data/imdb.sqllite


In [ ]:
print (imdb_db.rowCount('title_episodes'))    
print (imdb_db.rowCount('merged_title_ratings'))    
print (imdb_db.rowCount('merged_name_principals'))  
print (imdb_db.rowCount('title_episodes')['count'])

In [ ]:
pd.read_sql_query("select  nconst, ordering, category, job,\
                    primaryName, birthYear, deathYear, primaryProfession, \
                knownForTitles, isAlive, wiki from \
                ( SELECT * FROM merged_names_principals LIMIT 4 )  GROUP BY tconst ", imdbConn)

In [ ]:
pd.read_sql_query("select  count(1) from merged_names_principals", imdbConn).to_json(''.join([LESSON_DATA_FOLDER, 'x.json']))

In [ ]:
print (27172692/100000)

In [8]:
title_files =[]
LIMIT = 100000

for x in range(0, int(imdb_db.rowCount('merged_title_ratings')['count']/LIMIT) + 2):
    title_files.append({'index':x, 'limit':LIMIT, 'offset': LIMIT * x, 
                        'file':''.join([LESSON_DATA_FOLDER, 'titles.' , str(x), '.json'])})

#title_files[0]    
#title_files

In [12]:
import prep_data;
for title in title_files[45:]:   
    print ('Working on title : {}'.format(title))
    prep_data.prep_title (title)
    gc.collect()
    

Working on title : {'limit': 100000, 'offset': 2700000, 'index': 27, 'file': '../data/titles.27.json'}
Working on title : {'limit': 100000, 'offset': 2800000, 'index': 28, 'file': '../data/titles.28.json'}
Working on title : {'limit': 100000, 'offset': 2900000, 'index': 29, 'file': '../data/titles.29.json'}
Working on title : {'limit': 100000, 'offset': 3000000, 'index': 30, 'file': '../data/titles.30.json'}
Working on title : {'limit': 100000, 'offset': 3100000, 'index': 31, 'file': '../data/titles.31.json'}
Working on title : {'limit': 100000, 'offset': 3200000, 'index': 32, 'file': '../data/titles.32.json'}
Working on title : {'limit': 100000, 'offset': 3300000, 'index': 33, 'file': '../data/titles.33.json'}
Working on title : {'limit': 100000, 'offset': 3400000, 'index': 34, 'file': '../data/titles.34.json'}
Working on title : {'limit': 100000, 'offset': 3500000, 'index': 35, 'file': '../data/titles.35.json'}
Working on title : {'limit': 100000, 'offset': 3600000, 'index': 36, 'fil

KeyError: ('2016', 'occurred at index isAdult')

In [ ]:
def transform(df, key):
    mapping = FILE_MAPPINGS.getMapping(key)
    df.fillna(method='pad', inplace=True)
    if(mapping['to_replace']!= None):
        df.replace(to_replace=mapping['to_replace'],method='pad', inplace=True)
    
    converters = mapping['converters']
    if(converters!= None):
        cols = list(converters.keys())
        df[cols] = df[cols].apply(Utils.callFunction, **converters)
        
    return df

def prep_title(title):
    limit = title['limit']
    offset = title['offset']
    file = title['file']
    q_titles = "select  * from merged_title_ratings where titleType like 'tv%' LIMIT {limit} OFFSET {offset}".format(limit = limit, offset = offset)    
    df_titles = imdb_db.query(q_titles)
    df_titles = transform(df_titles, 'merged.title.ratings.sql')
    df_titles['imdb'] = df_titles['tconst'].apply(Utils.wikiLink, args = ('www.imdb.com/title/', ))
    
    #names['wiki'] = names['primaryName'].apply(Utils.wikiLink)
    
    q_names = 'SELECT * FROM merged_name_principals WHERE tconst IN ({tconsts})'.format(
        tconsts =  ','.join("'"+ tconst +"'" for tconst in df_titles['tconst']))
                    
    df_names  = imdb_db.query(q_names)
    df_names['imdb'] = df_names['nconst'].apply(Utils.wikiLink, args = ('www.imdb.com/name/', ))
    df_names = transform(df_names, 'merged.names.principals.sql')
    df_names['isAlive'] = df_names['deathYear'] == 0    
    df_names['wiki'] = df_names['primaryName'].apply(Utils.wikiLink)
    df_names[['lastName', 'firstName']] = df_names['primaryName'].apply(lambda x: pd.Series(str(x).lower().split(' ', 1)))
    
    dfg_names = df_names.groupby('tconst')
    
    q_episodes = 'SELECT * FROM title_episodes WHERE parentTconst IN ({tconsts})'.format(
        tconsts =  ','.join("'"+ tconst +"'" for tconst in df_titles['tconst']))
    df_episodes  = imdb_db.query(q_episodes)
    df_episodes = transform(df_episodes, 'title.episodes.sql')
    dfg_episodes = df_episodes.groupby(['parentTconst'])
    
    
    def funNames(row, df):       
        try:
            dic = df.get_group(row['tconst']).to_dict('records')
            #dic = dfg_episodes.get_group(row['tconst']).to_dict('records')
            return dic
        except:        
            return []
    
    def funSeasons(data):
        #print(data['tconst'])
        dic = {
            'seasonNumber':str(int(data['seasonNumber'].unique()[0])), 
            'episodes': data['tconst'],                        
             }
        #print(dic)
        return dic
    def funEpisodes(row, df):       
        try:            
            if(row['parentTconst'] == None):
                dic = df.get_group(row['tconst']).dropna(axis=0, how='any').groupby('seasonNumber').apply(funSeasons)                
                return dic
            else:
                return []
        except KeyError as kerr:
            return []          
        except Exception as ex:
            raise ex;
            return []  

    df_titles['crew'] = df_titles.apply(funNames, axis = 1, args= (dfg_names, ))
    df_titles['series'] = df_titles.apply(funEpisodes, axis = 1, args= (dfg_episodes, ))
    
    df_titles.to_json(file, orient='records')
    del [df_titles,df_names, dfg_names, df_episodes, dfg_episodes]
    gc.collect()

#prep_title(title_files[0])
for title in title_files[:1]:    
    #prep_title (title)
    #print (title)

In [ ]:
df_names[df_names['tconst'] == 'tt0000134']

In [ ]:
def fun(row):
    #print (row['tconst'])    
    try:
        dic = dfg_names.get_group(row['tconst']).to_dict('records')
        return dic
    except:        
        return [] 
    
df_titles['crew'] = df_titles.apply(fun, 1)

In [ ]:
df_titles.to_json(''.join([LESSON_DATA_FOLDER, 'titles.json']), orient='records')

In [ ]:
L = ['L','O','L']
', '.join("'"+ i +"'" for i in L)

In [ ]:
tconsts  = ['tt0774826','tt0041030', 'tt0041031']
query = 'select * from merged_title_ratings where tconst in ({tconsts})'.format(
    tconsts = ', '.join("'"+ i +"'" for i in tconsts))
pd.read_sql(query, con = imdbConn)

In [ ]:
df = imdb_db.query("select  * from title_episodes where parentTconst in('tt0041030', 'tt0041031')");
#df.groupby('parentTconst').get_group('tt0041030').describe()
def fun(data):
    
    return {
            'seasonNumber':''.join (data['seasonNumber'].unique()), 
            'tconsts':data['tconst'],            
            
             }
df.groupby(['parentTconst', 'seasonNumber']).apply(fun)

In [ ]:
df.groupby('parentTconst').get_group('tt0041031').dropna(axis=0, how='any').to_dict('list')

In [ ]:
tconsts  = ['tt0774826','tt0041030', 'tt0041031']
tconsts_string = ', '.join("'"+ i +"'" for i in tconsts)
#query = 'select * from merged_title_ratings where tconst in ({tconsts})'.format( tconsts = tconsts_string)
query = 'select {columns} from merged_title_ratings group by titleType'.format( columns = 'titleType, count(1)')
result = imdb_db.query(query);
result

In [ ]:
list1 = [1, 2, 3]
''.join(str(e) for e in list1)